In [12]:
# Create training videos
import cv2
import numpy as np
from time import sleep
import glob
import os
import sys

NUM_FRAMES = 100
SEQ_LENGTH = 20
TAKES_PER = 3
MOODS = ['EXCITED', 'RELAXED']
HIDDEN_SIZE = 256

In [1]:
# Record movies, get ready to act either excited or relaxed!
def capture(num_frames, path='out.avi', countdown=0):
    for i in reversed(range(max(0, countdown))):
        i = 'GO!' if i == 0 else '{}  '.format(i)
        sys.stdout.write("Download progress: %d%%   \r" % (progress) )
        sys.stdout.flush()
        sleep(1)

    # Create a VideoCapture object
    cap = cv2.VideoCapture(0)

    # Check if camera opened successfully
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")

    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

    print('Recording started')
    for i in range(num_frames):

        ret, frame = cap.read()

        if ret == True:     
            # Write the frame into the file 'output.avi'
            out.write(frame)


    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()
    print('Recording stopped')
    
for take in range(TAKES_PER):
    for mood in MOODS:
        path = 'data/{}{}.avi'.format(mood, take)
        print('Get ready to act:', mood)
        capture(NUM_FRAMES, path=path, countdown=3)

Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started


KeyboardInterrupt: 

In [15]:
# Create training data
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)
#model.summary()

def get_vectors(video_path):
    vidcap = cv2.VideoCapture(video_path)
    success, frame = vidcap.read()
    while success:
        #img = image.load_img(frame, target_size=(224, 224))
        #img_data = image.img_to_array(img)
        img_data = cv2.resize(frame,(224,224))
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        vgg16_feature = model.predict(img_data)
        yield vgg16_feature
        success,frame = vidcap.read()
    vidcap.release()

movies = []
for mood in MOODS:
    y = np.array([1,0]) if mood == 'EXCITED' else np.array([0,1])
    for video_path in glob.glob('video/{}*.avi'.format(mood)):
        print('preprocessing', video_path)
        vectors = [np.expand_dims(vec.flatten(), axis=0) for vec in get_vectors(video_path)]
        Xs = np.concatenate(vectors)
        movies.append([
            Xs,
            y
        ])

preprocessing video/EXCITED1.avi
preprocessing video/EXCITED0.avi
preprocessing video/EXCITED2.avi
preprocessing video/EXCITED3.avi
preprocessing video/RELAXED2.avi
preprocessing video/RELAXED3.avi
preprocessing video/RELAXED1.avi
preprocessing video/RELAXED0.avi


In [21]:
# create seqs
seq_Xs = []; seq_ys = []
for Xs, y in movies:
    for i in range(len(Xs) - SEQ_LENGTH):
        seq_Xs.append(Xs[i:i+SEQ_LENGTH])
        seq_ys.append(y)

print(seq_Xs[0].shape)
print(len(seq_Xs))
seq_Xs = np.concatenate([np.expand_dims(a, axis=0) for a in seq_Xs])
seq_ys = np.concatenate([np.expand_dims(a, axis=0) for a in seq_ys])

print('X/y dimensions:', seq_Xs.shape, seq_ys.shape)

(20, 25088)
640
X/y dimensions: (640, 20, 25088) (640, 2)


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from sklearn.model_selection import train_test_split

HIDDEN_SIZE_LSTM = 128
HIDDEN_SIZE_DENSE = 16

model = Sequential()
model.add(LSTM(HIDDEN_SIZE_LSTM, return_sequences=True, input_shape=(SEQ_LENGTH, seq_Xs.shape[2])))
model.add(LSTM(HIDDEN_SIZE_LSTM, return_sequences=True))
model.add(LSTM(HIDDEN_SIZE_LSTM))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

x_train, x_val, y_train, y_val = train_test_split(seq_Xs, seq_ys, random_state=42)
model.fit(x_train, y_train,
          batch_size=10, epochs=5,
          validation_data=(x_val, y_val))

Train on 480 samples, validate on 160 samples
Epoch 1/5
480/480 [==============================] - 39s 80ms/step - loss: 0.3076 - acc: 0.8271 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 2/5
480/480 [==============================] - 35s 73ms/step - loss: 5.4172e-04 - acc: 1.0000 - val_loss: 4.3486e-05 - val_acc: 1.0000
Epoch 3/5
480/480 [==============================] - 33s 69ms/step - loss: 3.9444e-05 - acc: 1.0000 - val_loss: 2.2087e-06 - val_acc: 1.0000
Epoch 4/5
480/480 [==============================] - 35s 72ms/step - loss: 9.8812e-06 - acc: 1.0000 - val_loss: 1.6838e-07 - val_acc: 1.0000
Epoch 5/5
480/480 [==============================] - 33s 70ms/step - loss: 4.7075e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000


In [35]:
seqi = 500
np.argmax(model.predict(seq_Xs[seqi:seqi+1]))

1